In [85]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import easyocr
import util
import pandas as pd
import re
from collections import deque
from tensorflow.keras.utils import to_categorical


In [86]:
model_cfg_path = './darknet-yolov3.cfg'
model_weights_path = './model.weights'
input_dir = './test_private/'
net = cv2.dnn.readNetFromDarknet(model_cfg_path, model_weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [87]:
def sort_by_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    else:
        return 0

filenames = os.listdir(input_dir)

sorted_filenames = sorted(filenames, key=sort_by_number)


In [88]:

predictions = []

for img_name in sorted_filenames:
    
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)
    
    H, W, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), True)

    net.setInput(blob)

    detections = util.get_outputs(net)

    bboxes = []
    class_ids = []
    scores = []

    for detection in detections:
        bbox = detection[:4]

        xc, yc, w, h = bbox
        bbox = [int(xc * W), int(yc * H), int(w * W), int(h * H)]

        bbox_confidence = detection[4]

        class_id = np.argmax(detection[5:])
        score = np.amax(detection[5:])

        bboxes.append(bbox)
        class_ids.append(class_id)
        scores.append(score)

    bboxes, class_ids, scores = util.NMS(bboxes, class_ids, scores)

    reader = easyocr.Reader(['en'])
    text = ''
    for bbox_, bbox in enumerate(bboxes):
        xc, yc, w, h = bbox

        license_plate = img[int(yc - (h / 1.5)):int(yc + (h / 1.5)), int(xc - (w / 1.5)):int(xc + (w / 1.5)), :].copy()

        img = cv2.rectangle(img,
                            (int(xc - (w / 2)), int(yc - (h / 2))),
                            (int(xc + (w / 2)), int(yc + (h / 2))),
                            (0, 255, 0),
                            2)

        license_plate_gray = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(license_plate_gray,(1,1),0)
        binary = cv2.threshold(blur, 180, 255,
                            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        _, license_plate_thresh = cv2.threshold(license_plate_gray, 64, 255, cv2.THRESH_BINARY_INV)

        output = reader.readtext(license_plate_gray, allowlist='0123456789')
        num_digits = 0
        for out in output:
            text += out[1]
            num_digits += len([c for c in out[1] if c.isdigit()])
            if num_digits >= 8:
                break
    
          
predictions.append({'image_name': img_name, 'predicted_text': text})
print(img_name, text)

901.jpg 1690339685
902.jpg 03
903.jpg 01372799
904.jpg 1236324
905.jpg 12078917
906.jpg 6752527201
907.jpg 757376560
908.jpg 177654712
909.jpg 54161
910.jpg 894187
911.jpg 118359630
912.jpg 71600538806
913.jpg 907530
914.jpg 110134969501
915.jpg 13165404
917.jpg 90337708
918.jpg 473951187
919.jpg 192696756
920.jpg 1416356603
921.jpg 1759335567
922.jpg 1613153230
923.jpg 131371612
924.jpg 793499778
925.jpg 881197703
926.jpg 9011397
927.jpg 169579044
928.jpg 14672755431
929.jpg 87638179
930.jpg 3733819
931.jpg 0
932.jpg 0874
933.jpg 90198483
934.jpg 351034142
935.jpg 180228033
936.jpg 9094636
937.jpg 1791
938.jpg 1355906
939.jpg 8763238179
940.jpg 913634293
941.jpg 184324868
942.jpg 4613293747
943.jpg 01
944.jpg 1703357645
945.jpg 72555043
946.jpg 1701116365
947.jpg 42341747
948.jpg 174074999
949.jpg 184185
950.jpg 14011714
951.jpg 14039240
952.jpg 429453973
953.jpg 180554161
954.jpg 162649407
955.jpg 1125445633
956.jpg 11375778459
957.jpg 266067
958.jpg 355370
959.jpg 1608734041
960.jpg

In [89]:
df = pd.DataFrame(predictions)
df.head(31)


,image_name,predicted_text
0,901.jpg,1690339685
1,902.jpg,03
2,903.jpg,01372799
3,904.jpg,1236324
4,905.jpg,12078917
5,906.jpg,6752527201
6,907.jpg,757376560
7,908.jpg,177654712
8,909.jpg,54161
9,910.jpg,894187


In [90]:
def Convert(string):
    la_list = []
    list1 = []
    list1[:0] = string
    while len(list1) > 7:
        del list1[-5]
    while len(list1) < 7:
        list1 = deque(list1)
        list1.appendleft('0')
        list1 = list(list1)

    for i in list1:
        test = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        new_val = int(i)
        test[new_val] = 1
        la_list.append(test)
  
    return la_list

In [91]:
df['encoded_text'] = df['predicted_text'].apply(Convert)

In [92]:
def encode_texts(df):
    output_df = pd.DataFrame(columns=['id', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    
    for _, row in df.iterrows():
        subarrays = np.array_split(row['encoded_text'], 7)
        
        for i, subarray in enumerate(subarrays):
            new_row = {'id': f"{'img'}_{row['image_name'].split('.')[0]}_{i+1}"}
            for j, digit in enumerate(subarray):
                new_row[str(j)] = digit
            
            output_df = output_df.append(new_row, ignore_index=True)
            
    for index,id in enumerate(output_df['id']):
        for i in range(1,10):
            output_df.loc[output_df['id']==id, f'{i}'] = output_df['0'][index][i]
        output_df.loc[output_df['id']==id, f'{0}'] = output_df['0'][index][0]              

    
    return output_df


In [93]:
output_df = encode_texts(df)

C:\Users\user\AppData\Local\Temp\ipykernel_1852\3042899133.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(new_row, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_1852\3042899133.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(new_row, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_1852\3042899133.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(new_row, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_1852\3042899133.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.app

In [94]:
output_df.to_csv("Submission3.csv", index=False)